In [2]:

# 2) 라이브러리 임포트
import os
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

# 3) GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# 4) CSV 파일 로드
#    - Colab에 파일을 업로드했거나 구글 드라이브에 마운트한 뒤 경로를 지정하세요.
# from google.colab import drive
# drive.mount('/content/drive')
csv_path = "/content/drive/MyDrive/movies_2015_plus_full_filtered.csv"
df = pd.read_csv(csv_path)

# 5) 사용할 컬럼만 추출 (imdb_id, tmdb_id 제외)
cols_meta = [
    "title",
    "release_date",
    "overview",
    "directors",
    "cast_top5",
    "genres",
    "production_companies",
    "production_countries"
]
df = df[cols_meta].copy()

# 6) 결측치(NaN) 처리
#    - 메타데이터가 누락된 경우 빈 문자열("")로 채웁니다.
for c in cols_meta:
    df[c] = df[c].fillna("").astype(str)

# 7) 입력용 텍스트 생성 함수
def make_input_text(row):
    """
    하나의 영화(row)에서 메타데이터 칼럼을 합쳐서 하나의 블록 텍스트로 만듭니다.
    토큰 길이가 384 토큰을 넘지 않도록 개략적으로 요약된 형태로 구성하세요.
    """
    parts = []
    # Title + Release Date
    if row["title"]:
        parts.append(f"Title: {row['title']}")
    if row["release_date"]:
        parts.append(f"Release Date: {row['release_date']}")

    # Overview (줄거리)
    if row["overview"]:
        parts.append(f"Overview: {row['overview']}")

    # Directors
    if row["directors"]:
        parts.append(f"Directors: {row['directors']}")

    # Cast (상위 5명 배우)
    if row["cast_top5"]:
        parts.append(f"Cast: {row['cast_top5']}")

    # Genres
    if row["genres"]:
        parts.append(f"Genres: {row['genres']}")

    # Production Companies
    if row["production_companies"]:
        parts.append(f"Production Companies: {row['production_companies']}")

    # Production Countries
    if row["production_countries"]:
        parts.append(f"Production Countries: {row['production_countries']}")

    # 각 파트는 " | " 로 구분
    return " | ".join(parts)

# 8) 모든 행에 대해 입력 텍스트 생성
tqdm.pandas()
df["input_text"] = df.progress_apply(make_input_text, axis=1)

# 9) SentenceTransformer 모델 로드
model_name = "beeformer/Llama-movielens-mpnet"
model = SentenceTransformer(model_name, device=device)
# 모델의 최대 시퀀스 길이를 384 로 제한 (기본값이 512인 경우)
model.max_seq_length = 384

# 10) 임베딩 생성
#     한꺼번에 메모리 과부하가 나지 않도록 batch_size를 적절히 설정하세요.
batch_size = 8  # GPU 리소스에 맞게 4,8,16 등을 조정
all_embeddings = []

for i in tqdm(range(0, len(df), batch_size), desc="Embedding batches"):
    batch_texts = df["input_text"].iloc[i : i + batch_size].tolist()
    embeddings = model.encode(
        batch_texts,
        batch_size=len(batch_texts),
        show_progress_bar=False,
        convert_to_numpy=True,
        max_length=384,
        truncation=True
    )
    all_embeddings.append(embeddings)

# 11) (N, 768) 크기로 합치기
all_embeddings = np.vstack(all_embeddings)
print("전체 임베딩 shape:", all_embeddings.shape)

# 12) 결과 저장
#     • embeddings.npy: (N, 768) NumPy 바이너리
#     • metadata.csv: index와 input_text를 포함한 최소 정보
np.save("embeddings.npy", all_embeddings)

# (선택) 나중에 매핑을 위해 input_text와 원래 row index를 함께 저장
df_meta = df[["input_text"]].copy()
df_meta.to_csv("metadata_for_embeddings.csv", index=True)

print("임베딩 파일과 메타데이터가 저장되었습니다:")
print("- embeddings.npy")
print("- metadata_for_embeddings.csv")


Device: cuda


  0%|          | 0/1019 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Embedding batches:   0%|          | 0/128 [00:00<?, ?it/s]

전체 임베딩 shape: (1019, 768)
임베딩 파일과 메타데이터가 저장되었습니다:
- embeddings.npy
- metadata_for_embeddings.csv
